In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

import lime

import matplotlib.pyplot as plt
import os 
import sys
import random
import warnings

import torch


warnings.filterwarnings("ignore")

SEED_VALUE = 100
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)
torch.cuda.manual_seed(SEED_VALUE)
torch.cuda.manual_seed_all(SEED_VALUE)


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

# city pollution data

In [3]:
time_series_data = pd.read_csv("./data/gams_indoor.csv")


DROP_ONEHOT = True
SEQ_LENGTH = 7


if DROP_ONEHOT:
  INPUT_DIM = 2
# else:
#   INPUT_DIM = 29

HIDDEN_DIM = 32
LAYER_DIM = 3


normalization_type = 'mean_std' # 'max', mean_std

In [4]:
import pandas as pd

# 데이터를 시간 순서대로 정렬합니다.
time_series_data = time_series_data.sort_values(by='ts')

# 훈련 데이터와 테스트 데이터의 기간을 결정합니다.
train_end_time = '2017-03-09 14:26:02'
test_start_time = '2017-03-09 14:27:02'

# 기준 시간을 기준으로 데이터를 분할합니다.
train_data = time_series_data[time_series_data['ts'] <= train_end_time]
test_data = time_series_data[time_series_data['ts'] >= test_start_time]

print("Train data:", train_data)
print("Test data:", test_data)

Train data:                          ts    co2  humidity  pm10  pm25  temperature    voc
0       2016-11-21 00:47:03  708.0     72.09  10.2   9.0        20.83  0.062
1       2016-11-21 00:48:03  694.0     70.95  10.9  10.1        21.01  0.062
2       2016-11-21 00:49:03  693.0     69.12  10.2   9.9        21.20  0.062
3       2016-11-21 00:50:03  692.0     68.83   9.6   9.6        21.37  0.062
4       2016-11-21 00:51:03  690.0     68.60   9.4   8.4        21.49  0.062
...                     ...    ...       ...   ...   ...          ...    ...
108074  2017-03-09 14:22:02  626.0     33.61  37.8  33.9        24.16  0.062
108075  2017-03-09 14:23:02  627.0     33.65  35.9  33.8        24.15  0.062
108076  2017-03-09 14:24:02  625.0     33.65  37.3  33.2        24.14  0.062
108077  2017-03-09 14:25:02  625.0     33.56  37.0  33.4        24.18  0.062
108078  2017-03-09 14:26:02  624.0     33.54  38.5  34.1        24.21  0.063

[108079 rows x 7 columns]
Test data:                          t

In [5]:
features = ['humidity','temperature']
SELECTED_COLUMN = "pm25" # ['co2','pm10','pm25','voc']
train_data = train_data[features + [SELECTED_COLUMN]]
test_data = test_data[features + [SELECTED_COLUMN]]

print(train_data)

        humidity  temperature  pm25
0          72.09        20.83   9.0
1          70.95        21.01  10.1
2          69.12        21.20   9.9
3          68.83        21.37   9.6
4          68.60        21.49   8.4
...          ...          ...   ...
108074     33.61        24.16  33.9
108075     33.65        24.15  33.8
108076     33.65        24.14  33.2
108077     33.56        24.18  33.4
108078     33.54        24.21  34.1

[108079 rows x 3 columns]


In [6]:
class GamsIndoor(torch.utils.data.Dataset):
  def __init__(self, data, selected_column, seq_length):
    self.selected_column = selected_column
    self.data = data 
    self.seq_length = seq_length

  def __getitem__(self, idx):
    seq = self.data[features][idx:idx+self.seq_length]
    target = self.data[SELECTED_COLUMN][idx:idx+self.seq_length]
    return torch.tensor(seq.values, dtype=torch.float32), torch.tensor(target.values, dtype=torch.float32)

  def __len__(self):
    return len(self.data) - self.seq_length


data pre-processing

In [7]:
# function that implement the look_ahead mask for masking future time steps. 
def create_look_ahead_mask(size, device):
    mask = torch.ones((size, size), device=device)
    mask = torch.triu(mask, diagonal=1)
    return mask  # (size, size)

# Model fitting

In [8]:
from models import *
from torch import nn
from torch import optim
import torch

device="cuda"

# Evaluation
def evaluation(testLoader, model, model_name, LUR, SELECTED_COLUMN, mask=False):
    model.to(device)
    model.eval()
    mse_list = []
    total_se = 0.0
    total_pe = 0.0
    total_valid = 0.0

    for x_val, y_val in testLoader:
        x_val, y_val = [t.cuda().float() for t in (x_val, y_val)]
        
        if mask:
            masking = create_look_ahead_mask(x_val.shape[1], device)
            out, _ = model(x_val.to(device), masking)
        else:
            out = model(x_val.to(device))

        if LUR :
            ytrue = y_val.squeeze().cpu().numpy()
            ypred = out.squeeze().cpu().detach().numpy()
        else:
            ytrue = y_val.squeeze().cpu().numpy()
            ypred = out.squeeze().cpu().detach().numpy()
        true_valid = np.isnan(ytrue) != 1
        ytrue = ytrue[true_valid] #np.nan_to_num(ytrue, 0)
        ypred = ypred[true_valid]

        se = (ytrue - ypred)**2 # np.square(ytrue - ypred)
        pe = np.abs((ytrue - ypred) / (ytrue + 0.0001))
        total_se += np.sum(se)
        total_pe += np.sum(pe)
        total_valid += np.sum(true_valid)

    eval_mse = total_se / total_valid # np.mean(se) # 
    eval_mape = total_pe / total_valid # np.mean(pe) # 
    print('valid samples:', total_valid)
    print('Eval MSE: ', eval_mse)
    print('Eval RMSE: {}: '.format(SELECTED_COLUMN), np.sqrt(eval_mse))
    print('Eval MAPE: {}: '.format(SELECTED_COLUMN), eval_mape*100)
    
    return eval_mse, eval_mape*100


# Train
def train(trainLoader, testLoader, model, model_name, SELECTED_COLUMN, mask=False, LUR=False, l1=False, l2=False):

    lr = 0.001
    n_epochs = 10   
    
    model.to(device)

    criterion = nn.MSELoss()
    
    # LUR
    if LUR:
        print("set l1,l2 loss")
        l1_lmbda = 0.01
        l1_lmbda = torch.FloatTensor([l1_lmbda]).cuda()
        l1_reg = torch.tensor(0., requires_grad=True).to(device)
        l2_lmbda = 0.01
        l2_lmbda = torch.FloatTensor([l2_lmbda]).cuda()
        l2_reg = torch.tensor(0., requires_grad=True).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=lr)
            
    # DL
    else:
        print("set SoftDTW loss")
        lmbda = 0.5
        dtw_loss = SoftDTW(use_cuda=True, gamma=0.1)
        opt = torch.optim.Adam(model.parameters(), lr=lr)
    
      
    print('Start ' + model_name + ' training')
    best_mse = 2000.0
    mape = 2000.0
    best_model = None
    

    for epoch in range(1, n_epochs + 1):
        # print("start epoch")
        epoch_loss = 0
        batch_idx = 0
        bar = tqdm(trainLoader)
        # print("start tqdm")
        model.train()
        # print("start batch before")
        for x_batch, y_batch in bar:
            # print("start batch")
            model.train()
            x_batch = x_batch.cuda().float()
            y_batch = y_batch.cuda().float()

            # print("start mask")
            if mask==True:
                masking = create_look_ahead_mask(x_batch.shape[1], device)
                out, _ = model(x_batch.to(device), masking)
            else :
                out = model(x_batch.to(device))
            opt.zero_grad()
            
            # print("start loss")
            if LUR:
                # LASSO
                if l1==True and l2==False:
                    l1_reg = torch.norm(model.linear.weight, p=1)
                    loss = criterion(out, y_batch.unsqueeze(2)) + l1_lmbda * l1_reg
                # Ridge
                elif l1==False and l2==True:
                    l2_reg = torch.norm(model.linear.weight, p=2)
                    loss = criterion(out, y_batch.unsqueeze(2)) + l2_lmbda * l2_reg
                # Elastic
                elif l1==True and l2==True:
                    l1_reg = torch.norm(model.linear.weight, p=1)
                    l2_reg = torch.norm(model.linear.weight, p=2)
                    loss = criterion(out, y_batch.unsqueeze(2)) + l1_lmbda * l1_reg + l2_lmbda * l2_reg
                # OLS
                else:
                    loss = criterion(out, y_batch.unsqueeze(2))
            else:
                # print(out.shape)
                # print(y_batch.shape)
                # print(y_batch.unsqueeze(2).shape)
                loss = criterion(out, y_batch.unsqueeze(2)) + lmbda * dtw_loss(out.cuda(),y_batch.unsqueeze(2).cuda()).mean()

            epoch_loss = (epoch_loss*batch_idx + loss.item())/(batch_idx+1)
            loss.backward(retain_graph=True)
            opt.step()

            bar.set_description(str(epoch_loss))
            batch_idx += 1

        eval_mse, eval_mape = evaluation(testLoader, model, model_name, LUR, SELECTED_COLUMN, mask)
        

        if eval_mse < best_mse:
            best_model = deepcopy(model)
            best_mse = eval_mse
            mape = eval_mape
            torch.save(best_model.state_dict(), "./save_gams/"+SELECTED_COLUMN+"/"+model_name+".pth")
      
    print(model_name)   
    print("Best MSE :", best_mse)
    print("RMSE :", np.sqrt(best_mse))
    print("MAPE :", mape)
    print()


In [11]:
from models import *
from loss_utils import *


train_data = GamsIndoor(train_data, SELECTED_COLUMN, SEQ_LENGTH)
test_data = GamsIndoor(test_data, SELECTED_COLUMN, SEQ_LENGTH)
trainLoader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4, worker_init_fn=seed_worker, generator=g)
testLoader = DataLoader(test_data, batch_size=4096, shuffle=False, num_workers=4, worker_init_fn=seed_worker, generator=g)

In [10]:
# RNN
RNNmodel = RNN(1, INPUT_DIM, HIDDEN_DIM, LAYER_DIM).to(device)
train(trainLoader, testLoader, RNNmodel, "RNN", SELECTED_COLUMN)
# LSTM
LSTMmodel = LSTM(1, INPUT_DIM, HIDDEN_DIM, LAYER_DIM).to(device)
train(trainLoader, testLoader, LSTMmodel, "LSTM", SELECTED_COLUMN)
# # BiLSTM
# # BiLSTMmodel = LSTM(1, INPUT_DIM+1, HIDDEN_DIM, LAYER_DIM, bidirectional=True).to(device)
# # train(trainLoader, testLoader, BiLSTMmodel, "BiLSTM", SELECTED_COLUMN)
# # TransLSTM
# TransLSTMmodel = TransLSTM(num_layers=3, D=16, H=5, hidden_mlp_dim=32, inp_features=2, out_features=1, dropout_rate=0.2, LSTM_module = LSTM(1, INPUT_DIM, HIDDEN_DIM, LAYER_DIM, bidirectional = False).to(device), attention_type='regular').to(device) # cosine_square, cosine, regular # 6L, 12H
# train(trainLoader, testLoader, TransLSTMmodel, "TransLSTM", SELECTED_COLUMN, mask=True)
# Transformer
Transmodel = Transformer(num_layers=6, D=16, H=10, hidden_mlp_dim=32, inp_features=2, out_features=1, dropout_rate=0.1, attention_type='regular', SL=SEQ_LENGTH).to(device) # cosine_square, cosine, regular # 6L, 12H
train(trainLoader, testLoader, Transmodel, "Transformer", SELECTED_COLUMN, mask=True)
# CosFormer
TransCosModel = Transformer(num_layers=6, D=16, H=10, hidden_mlp_dim=32, inp_features=2, out_features=1, dropout_rate=0.1, attention_type='cosine', SL=SEQ_LENGTH).to(device) # cosine_square, cosine, regular # 6L, 12
train(trainLoader, testLoader, TransCosModel, "CosFormer", SELECTED_COLUMN, mask=True)
# CosSquareFormer
TransCosSquare = Transformer(num_layers=6, D=16, H=10, hidden_mlp_dim=32, inp_features=2, out_features=1, dropout_rate=0.1, attention_type='cosine_square', SL=SEQ_LENGTH).to(device) # cosine_square, cosine, regular # 6L, 12H
train(trainLoader, testLoader, TransCosModel, "CosSquareFormer", SELECTED_COLUMN, mask=True)


set SoftDTW loss
Start Transformer training


731.2181280801242: 100%|██████████| 3378/3378 [02:18<00:00, 24.46it/s]


valid samples: 189091.0
Eval MSE:  86.75442247383535
Eval RMSE: pm25:  9.314205412907498
Eval MAPE: pm25:  103.83594499886959


675.4246341937954: 100%|██████████| 3378/3378 [02:25<00:00, 23.30it/s]


valid samples: 189091.0
Eval MSE:  87.50575384338758
Eval RMSE: pm25:  9.354451017744847
Eval MAPE: pm25:  105.09346204753929


...

valid samples: 189091.0
Eval MSE:  88.09119749221274
Eval RMSE: pm25:  9.385691103600882
Eval MAPE: pm25:  106.03160730151092


675.40104391167: 100%|██████████| 3378/3378 [03:47<00:00, 14.88it/s]  


valid samples: 189091.0
Eval MSE:  89.17515607564611
Eval RMSE: pm25:  9.443259822521359
Eval MAPE: pm25:  107.68552947059882


675.3747500548901: 100%|██████████| 3378/3378 [03:45<00:00, 14.96it/s]


valid samples: 189091.0
Eval MSE:  88.6828276464771
Eval RMSE: pm25:  9.417156027510487
Eval MAPE: pm25:  106.94663982057581
CosSquareFormer
Best MSE : 85.81530255273915
RMSE : 9.263654924096599
MAPE : 102.16425846219546



In [12]:
OLS = LinearRegression(input_dim=2)
train(trainLoader, testLoader, OLS, "OLS", SELECTED_COLUMN, LUR=True)
LASSO = LinearRegression(input_dim=2)
train(trainLoader, testLoader, LASSO, "LASSO", SELECTED_COLUMN, LUR=True, l1=True)
Ridge = LinearRegression(input_dim=2)
train(trainLoader, testLoader, Ridge, "Ridge", SELECTED_COLUMN, LUR=True, l2=True)
Elastic = LinearRegression(input_dim=2)
train(trainLoader, testLoader, Elastic, "Elastic", SELECTED_COLUMN, LUR=True, l1=True, l2=True)

set l1,l2 loss
Start OLS training


159.0960725323563: 100%|██████████| 3378/3378 [00:36<00:00, 91.88it/s]  


valid samples: 189091.0
Eval MSE:  89.4088844789017
Eval RMSE: pm25:  9.455627133030452
Eval MAPE: pm25:  107.30665011737072


155.38744909061822: 100%|██████████| 3378/3378 [00:34<00:00, 96.73it/s] 


valid samples: 189091.0
Eval MSE:  90.60020505999756
Eval RMSE: pm25:  9.518413999191123
Eval MAPE: pm25:  110.61226338690761


...

valid samples: 189091.0
Eval MSE:  89.55077912222157
Eval RMSE: pm25:  9.463127343654506
Eval MAPE: pm25:  110.84618370338224


153.9041384679458: 100%|██████████| 3378/3378 [00:43<00:00, 78.53it/s] 


valid samples: 189091.0
Eval MSE:  89.05481553855022
Eval RMSE: pm25:  9.43688590259256
Eval MAPE: pm25:  110.2662463319579


153.71889442660677: 100%|██████████| 3378/3378 [00:43<00:00, 77.78it/s]


valid samples: 189091.0
Eval MSE:  87.13915059415837
Eval RMSE: pm25:  9.334835327640139
Eval MAPE: pm25:  107.61487483209142
Elastic
Best MSE : 87.13915059415837
RMSE : 9.334835327640139
MAPE : 107.61487483209142

